<a href="https://colab.research.google.com/github/luciaPi/Neural-networks-course/blob/main/5Trening_rozdeleny_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tréning neurónovej siete**
**Dataset Študenti**

https://www.kaggle.com/datasets/rkiattisak/student-performance-in-mathematics

In [ ]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import torch.utils.data as data_utils
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import copy

#### **1. Príprava dát**

In [ ]:
#!ls /root/.cache/kagglehub/datasets/rkiattisak/student-performance-in-mathematics/versions/3

import kagglehub

# Download latest version
path = kagglehub.dataset_download("rkiattisak/student-performance-in-mathematics")

print("Path to dataset files:", path)
data = pd.read_csv(path+"/exams.csv")

# zmenime "lunch" na 0-1 hodnoty
data['lunch'] = data['lunch'].replace({'free/reduced': 1, 'standard': 0})
# upravime aj "gender" a "parental level of education"
data['gender'] = data['gender'].replace({'female': 1, 'male': 0})
data['parental level of education'] = data['parental level of education'].replace({'some high school': 0,'high school':0,'some college':1,"associate's degree":2,"bachelor's degree":3,"master's degree":4})
# zmenime na 0-1 hodnoty stlpec "test preparation course"
data['test preparation course'] = data['test preparation course'].replace({'completed': 1, 'none': 0})
# vymazeme stlpec "race/ethnicity"
data =data.drop(columns=['race/ethnicity'])

# vstupy
input_name1 = 'test preparation course'
input_name2 = 'reading score'
input_name3 = 'writing score'
inputs = data[[input_name1,input_name2,input_name3]].copy()

# vystup
output_name = 'math score'
labels = data[output_name].copy()

data

**Rozdelenie trénovacích dát**
* Train set - ???
* Valid set - 10%
* Test set -  20%

In [ ]:
X_trainval, X_test, y_trainval, y_test = train_test_split(inputs, labels, test_size=???)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=???)

In [ ]:
# data normalization
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train, y_train = np.array(X_train), np.array(y_train).astype(float)
X_val, y_val = np.array(X_val), np.array(y_val).astype(float)
X_test, y_test = np.array(X_test), np.array(y_test).astype(float)

# upravime data na tenzory
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_val = torch.from_numpy(y_val).float()
y_test = torch.from_numpy(y_test).float()

print("Train:", X_train.shape, y_train.shape)
print("Valid:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)
print("Example:", X_train[:5], y_train[:5])

**Príprava DataLoadera**

Samostaný DataLoader pre **train**, **valid** a **test**.

**Valid** a **test** DataLoader majú *epoch=1* a *shuffle=False*.

In [ ]:
batch_size = 32
# Train set
train_tensor = data_utils.TensorDataset(X_train, y_train)
train_dataloader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)
# Valid set
valid_tensor = data_utils.TensorDataset(X_val, y_val)
valid_dataloader = data_utils.DataLoader(dataset = valid_tensor, batch_size = 1, shuffle = False)
# Test set
test_tensor = data_utils.TensorDataset(X_test, y_test)
test_dataloader = data_utils.DataLoader(dataset = test_tensor, batch_size = 1, shuffle = False)

#### **2. Príprava neurónovej siete**

In [ ]:
# prioritne budeme trenovat na GPU karte
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# postavime neuronovu siet
class StudentiModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(3, 3)  # torch.nn.Linear(in_features, out_features)
        self.linear2 = nn.Linear(3, 2)
        self.linear3 = nn.Linear(2, 1)

    def forward(self, x):  # x = vstup
        y123 = self.linear1(x)
        y45 = self.linear2(y123)
        y6 = self.linear3(y45)
        return y6

# vyrvorime neuronovu siet
model = StudentiModel().to(device).eval()
model

#### **3. Tréning neurónovej siete**

In [ ]:
model.train() # train() - nastavime model do treningoveho modu (opakom je eval() - evaluacny/netreningovy mod)

# optimizer - optimalizuje parametre (vahy, bias) siete
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.9)

# Loss funkcia = chyba siete - trening sa ju snazi zlepsit
criterion = nn.MSELoss() # MSE=mean squarred error (priemerna stvorcova chyba)

# pocet epoch
epochs = 100

* Nájdite rozdiel medzi týmto tréningovým cyklom a cyklom z colab notebooku 3 alebo 4.

In [ ]:
print("Start training")

loss_stats_MSE = {
    'train': [],
    "val": []
}
loss_stats_MAE = {
    'train': [],
    "val": []
}

best_model = None
best_valid_MSE = 10000

for epoch in range(epochs): # pocas jednej epochy prejdeme cely dataset

    # inicializujeme pre treningove statistiky
    running_loss_mse_train = 0.0
    running_mae_train = 0.0

    # v cykle prejdeme vsetky data
    for i, data in enumerate(train_dataloader):
        inputs, labels = data

        # reshape data
        inputs = inputs.view(-1,3)
        labels = labels.view(-1,1)

        # TRAINING
        # restartujeme optimizer
        optimizer.zero_grad()

        # vypocitame vystup siete pre aktualny batch dat
        y = model(inputs)

        # priemerna absolutna chyba (MAE=mean absolute error)
        MAE = abs(labels-y).mean()
        running_mae_train += MAE.item()
        # chyba siete - trening sa ju snazi zlepsit (MSE=mean squarred error)
        loss = criterion(y, labels)

        # uprava parametrov siete podla aktualnej chyby (loss)
        loss.backward()
        optimizer.step()

        running_loss_mse_train += loss.item()

        running_loss_mse_val = 0.0
        running_mae_val = 0.0
        # VALIDATION
        with torch.no_grad():     # model sa netrenuje
          model.eval()  # evaluacny mod modelu - netrenuje sa
          for X_val_batch, y_val_batch in valid_dataloader:
              X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)

              # vypocet vystupov neuronovej siete pre validacne data
              y_val_pred = model(X_val_batch)
              # vypocet chyby siete pre validacne data
              val_loss = criterion(y_val_pred, y_val_batch.unsqueeze(1))
              running_loss_mse_val += val_loss.item()
              # priemerna absolutna chyba (MAE=mean absolute error)
              MAE = abs(y_val_pred-y_val_batch.unsqueeze(1)).mean()
              running_mae_val += MAE.item()

    # priemerna chyba za celu epochu
    MSE_train = running_loss_mse_train/len(train_dataloader)
    loss_stats_MSE['train'].append(MSE_train)
    MSE_val = running_loss_mse_val/len(valid_dataloader)
    loss_stats_MSE['val'].append(MSE_val)
    MAE_train = running_mae_train/len(train_dataloader)
    loss_stats_MAE['train'].append(MAE_train)
    MAE_val = running_mae_val/len(valid_dataloader)
    loss_stats_MAE['val'].append(MAE_val)

    print('epoch {0}: TRAIN MSE={1:0.2f} MAE={2:0.2f}    VALID MSE={3:0.2f} MAE={4:0.2f}'.format(epoch+1,
    MSE_train, MAE_train, MSE_val, MAE_val))

    # ulozenie najlepsieho modelu
    if (MSE_val <= best_valid_MSE):
      best_valid_MSE = MSE_val
      best_model = copy.deepcopy(model)

print("Finished training")

* Pridali sme **validačnú časť**, čiže už sa počas tréningu neukladá len chyba trénovacích dát, ale aj validačných.
* Počas validácie sa model **netrénuje**, je v evaluačnom móde.
* Priebežne si ukladáme model s **najlepšou validačnou Loss**. Tento model by mal najlepšie vedieť **generalizovať** a teda ho použijeme aj na testovanie.

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(10,4),constrained_layout=True)
axes[0,0].plot(loss_stats_MSE['train'], label='Training Loss MSE')
axes[0,1].plot(loss_stats_MAE['train'], label='Train MAE')
axes[0,0].set_xlabel('Epochs')
axes[0,1].set_xlabel('Epochs')
axes[0,0].set_title('Training Loss MSE')
axes[0,1].set_title('Train MAE')
axes[1,0].plot(loss_stats_MSE['val'], label='Valid Loss MSE')
axes[1,1].plot(loss_stats_MAE['val'], label='Valid MAE')
axes[1,0].set_xlabel('Epochs')
axes[1,1].set_xlabel('Epochs')
axes[1,0].set_title('Valid Loss MSE')
axes[1,1].set_title('Valid MAE')
plt.show()

#### **4. Ohodnotenie siete po tréningu**
* Na ohodnotenie modelu použijeme testovacie dáta.

**Ohodnotenie testovacích dát**

In [ ]:
# vystup celeho datasetu
y_pred_list = []
with torch.no_grad():
    # nastavime model do evaluacneho (netreningoveho modu)
    best_model.eval()
    for X_batch, _ in test_dataloader:
        X_batch = X_batch.to(device)
        y_test_pred = best_model(X_batch)
        y_pred_list.append(y_test_pred.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

# priemerna absolutna chyba (MAE=mean absolute error)
MAE = abs(y_pred_list-y_test.numpy()).mean().item()

# priemerna stvorcova chyba (MSE=mean squarred error)
MSE = ((y_pred_list-y_test.numpy())**2).mean().item()

print("MAE:",MAE)
print("MSE:",MSE)